In [2]:
import os
# from dotenv import load_dotenv

# load_dotenv()


# OPENAI_KEY = os.getenv("OPENAI_KEY")
# MODEL = "gpt-3.5-turbo"
# MODEL = "mixtral:8x7b"
MODEL = "llama3"

In [5]:
# from langchain_openai.chat_models import ChatOpenAI
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    # model = ChatOpenAI(api_key=OPENAI_KEY, model=MODEL)
    # embedding = OpenAIEmbeddings()
    pass
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model="llama3")
    
model.invoke("Tell me a joke")

"Here's one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!\n\nHope that made you smile!"

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain=model | parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?"

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("DD-CAREM25CO-3-r0_b.pdf")
pages = loader.load_and_split()


In [8]:
from langchain.prompts import PromptTemplate

template = """
Answer in spanish the question based on the context below. If you cant't
answer the quiestion, reply "I dont't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))




Answer in spanish the question based on the context below. If you cant't
answer the quiestion, reply "I dont't know".

Context: Here is some context

Question: Here is a question



In [9]:
chain = prompt | model | parser 

In [10]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question']}

In [11]:
chain.invoke(
    {
        "context": "The name I was given was Santiago",
        "question": "What's my name?"
    }
)

'Tu nombre es Santiago.'

In [13]:
from langchain_community.vectorstores import DocArrayInMemorySearch



vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings)

d:\Users\juanp_schamun\AppData\Local\anaconda3\envs\app_chat_doc\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [14]:
retriever = vectorstore.as_retriever()
retriever.invoke("catalogacion")

[Document(metadata={'source': 'DD-CAREM25CO-3-r0_b.pdf', 'page': 7}, page_content='FO-PR-002 r 3  \nCNEA  Glosario Coordinación y P reservación de la \nInformación y Operación  DD-CAREM25CO -3-\nC0021 Rev.:0  \nPágina:  8 de 9 \n \nEste documento es copia no controlada cuando está impreso. Es v álido el documento disponible en la base de datos CAREM  \n ARCHIVO ELECTRÓNICO: DD-CAREM25CO -3-r0_b.docx   Preservación de la Información y Operación.  \nSupplier for \nApproval  (Para \nAprobación De \nProveedores ): Estado del ciclo de vida del Contexto Proveedor de un Objeto que fue \nrevisado y su grado de maduración está  en una etapa avanzada \ncomo para ser aprobado por el Proveedor y puesto a disposición de \nCNEA.  \nEl Rol “Aprobador Proveedor” es el que puede ejecutar acciones en \neste estado (por ej. promoverlo, subir comentarios, etc., salvo \nmodificarlo).  \nSupplier InCheck  \n(en Revisión  de \nProveedores ): Estado del ciclo de vida del Contexto Proveedor, de un objeto, dond

In [15]:
from operator import itemgetter
chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
| parser
)

chain.invoke({"question": "Que es Documento con comentarios"})

'En el contexto descrito, el Documento con comentarios se refiere a un documento (por ejemplo, archivo Word) elaborado por un Revisor de un AT para realizar comentarios sobre un Documento de Ingeniería durante la revisión. Es complementario a los Comentarios dentro del sistema ENOVIA y permite agregar más detalles con diferentes formatos, como imágenes o tablas.'

In [16]:
questions = [
    "Que es PLM",
    "Que es Documento con comentarios",
    "Que son los estados finales",
    "Que son los estados de maduracion",
    "Que es una instancia",
    "Que es un nodo de corte",
    "Qué es 'Supplier  InCheck'?"
]

# for question in questions:
#         print(f"Question: {question}\n")
#         print(f"Aswer: {chain.invoke({'question': question})}")


In [17]:
for s in chain.stream({"question": "Qué es 'Supplier  InCheck'?"}):
    print(s, end="", flush=True)

La respuesta a esta pregunta en español sería:

"Proveedor InCheck no está definido en este contexto. La palabra "Proveedor" se refiere al proveedor externo que genera un objeto en CATIA V5/ENOVIA V5, mientras que "InCheck" no parece estar relacionado con este término."

In [18]:
chain.batch([{"question": q} for q in questions])

['En este contexto, no se menciona explícitamente el término "PLM", pero sí se describe el sistema ENOVIA V5 y su funcionalidad de gestión de productos (Product Lifecycle Management).\n\nSi bien no está directamente relacionado con la pregunta, podemos inferir que PLM (Product Lifecycle Management) se refiere a la gestión integral del ciclo de vida de un producto, desde su diseño hasta su desecho, incluyendo todas las etapas y actividades involucradas en ese proceso. En este sentido, el sistema ENOVIA V5 sería un ejemplo de una plataforma PLM que facilita la colaboración y el seguimiento del ciclo de vida de los productos.\n\nSi necesitas una respuesta más directa, podrías mencionar: "La PLM (Product Lifecycle Management) es el conjunto de procesos, tecnologías y estrategias para gestionar integralmente el ciclo de vida de un producto, desde su diseño hasta su desecho."',
 'Según el contexto, un "Documento con comentarios" se refiere a un documento (por ejemplo, archivo Word) elaborado